In [1]:
# Set up Notebook
% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm


# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [2]:
train_data = pd.read_csv('trainhh.csv')
test_data = pd.read_csv('testhh.csv')
labels = ['logFCS', 'RCSI', 'HDDS']

In [3]:
x_train=train_data.drop(labels, axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_test=test_data.drop(labels, axis=1)
x_test = x_test.drop(x_test.columns[0], axis=1)

x_train.head()

,clust_L12raincytot,clust_L12day1rain,clust_L12maxdays,clust_floodmax,clust_cells_own,clust_price,clust_thinn,clust_roof,clust_hhsize,clust_hh_age,clust_hh_gender,clust_asset,clust_dist_road,clust_dist_admarc,clust_percent_ag,clust_nutri_reten_constrained,clust_elevation,ipc_lag1,ipc_lag12
0,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
1,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
2,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
3,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN
4,1090.0613,43,17,0.0,0.125,3.824215,0.359375,0.125,5.4375,41.625,1.25,-0.327686,1.395625,5.176875,0.6,0.0,1311.875,1.0,NaN


In [4]:
y_train = train_data[labels]
y_test = test_data[labels]
y_test.head()

,logFCS,RCSI,HDDS
0,3.044522,22,2
1,3.637586,16,6
2,3.784190,23,4
3,4.182050,0,6
4,3.526361,39,4


In [9]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [20]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2,interaction_only =True)
x_train = poly.fit_transform(x_train)
x_test =  poly.fit_transform(x_test)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train[labels[0]])

# Compute model predictions for test data
pred = model.predict(x_test)

actual = y_test[labels[0]]
r2_linear = stats.pearsonr(actual, pred)[0] ** 2
r2_linear

0.00016583798514501673

In [ ]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True)

parameters = {'max_depth':np.arange( 1,8, 1 ).tolist(), 'min_samples_leaf':np.arange( 1, 6, 1 ).tolist()}
clf = GridSearchCV(rfc, parameters,cv=6, n_jobs= 4, iid = True,  refit= True,pre_dispatch= '2*n_jobs')
clf.fit(x_train, y_train[labels[0]])

# Fit estimator and display score

# Regress on test data
pred = clf.predict(x_test)

actual = y_test[labels[0]]
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

In [8]:
from sklearn.ensemble import ExtraTreesRegressor

# Create Regressor with default properties
extratree = ExtraTreesRegressor(random_state=0)

# Fit estimator and display score
extratree = extratree.fit(x_train, y_train[labels[0]])
# Regress on test data
pred = extratree.predict(x_test)

actual = y_test[labels[0]]
r2_et = stats.pearsonr(actual, pred)[0] ** 2
r2_et




0.14381231563503621

In [11]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(warm_start=False,random_state =0, validation_fraction=0.1,learning_rate='adaptive')
# Fit estimator and display score
mlp = mlp.fit(x_train, y_train[labels[0]])
pred = mlp.predict(x_test)

actual = y_test[labels[0]]
r2_mlp= stats.pearsonr(actual, pred)[0] ** 2
r2_mlp

0.032189761784622579

In [12]:
from sklearn.svm import SVR

# Create Regressor with default properties
svr = SVR(kernel='linear',degree =4)

# Fit estimator and display score
svr = svr.fit(x_train, y_train[labels[0]])
pred = svr.predict(x_test)

actual = y_test[labels[0]]
r2_svr= stats.pearsonr(actual, pred)[0] ** 2
r2_svr


0.13853357718871265

In [13]:
from sklearn.gaussian_process import  GaussianProcessRegressor

# Import our kernels
from sklearn.gaussian_process.kernels import Matern, WhiteKernel



# Define custom kernel (Matern + noise)
krnl = 2. + Matern(length_scale=1, nu=1.8) + WhiteKernel(noise_level=10)

# Create Regressor with specified properties
gpr = GaussianProcessRegressor(kernel=krnl, random_state=23)

# Fit estimator and display score
gpr= gpr.fit(x_train, y_train[labels[0]])
pred = gpr.predict(x_test)
actual = y_test[labels[0]]
r2_gpr= stats.pearsonr(actual, pred)[0] ** 2
r2_gpr



KeyboardInterrupt: 

In [13]:

from sklearn.linear_model import Ridge

# Define different alpha values for different fits
alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge = Ridge(alpha=20,fit_intercept=False)

ridge.fit(x_train, y_train[labels[0]])
pred = ridge.predict(x_test)

actual = y_test[labels[0]]
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

0.03528066763741608

In [14]:
from sklearn.linear_model import BayesianRidge
bridge = BayesianRidge(compute_score=True)
bridge.fit(x_train, y_train[labels[0]])
pred = bridge.predict(x_test)

actual = y_test[labels[0]]
r2_bridge= stats.pearsonr(actual, pred)[0] ** 2
r2_bridge

0.11664112931475472

In [15]:
from sklearn.linear_model import Lasso
alpha = [1E-4, 1E-2, 0.1, 1.0]

ls = Lasso(alpha=alpha[2], fit_intercept=True)


ls = ls.fit(x_train, y_train[labels[0]])
pred = ls.predict(x_test)

actual = y_test[labels[0]]
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

0.09700276711512458

In [18]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

alpha = [1E-4, 1E-2, 0.1, 1.0]


en = ElasticNet()

parameters = {'l1_ratio':np.arange( 0.1, 0.4, 0.05 ).tolist(), 'alpha':np.arange( 0.1, 0.3, 0.05 ).tolist()}
clf = GridSearchCV(en, parameters,cv=5, n_jobs= 4, iid = True,  refit= True,pre_dispatch= '2*n_jobs')
clf.fit(x_train, y_train[labels[0]])

pred = clf.predict(x_test)

actual = y_test[labels[0]]
r2_en= stats.pearsonr(actual, pred)[0] ** 2
r2_en


0.14004949890806

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

# Create Regressor with default properties
gbr = GradientBoostingRegressor(random_state=0)

gbr.fit(x_train, y_train[labels[0]])
pred = gbr.predict(x_test)

actual = y_test[labels[0]]
r2_gbr= stats.pearsonr(actual, pred)[0] ** 2
r2_gbr


0.13047944002981701